### ==============================================================================
### SCRIPT NAME: 03_CS_Modeling
### PURPOSE: Develop model
### PACKAGES NEEDED: numpy, pandas, matplotlib, seaborn, sklearn
### ==============================================================================

In this notebook you will be introduced to DATA MODELING in Python.

Whenever in doubt you can always refer to these resources:

    help([function name]): Provides a detailed description of the function/
    online pandas documentation: https://pandas.pydata.org/pandas-docs/stable/?v=20200107131408
    online matplotlib documentation: https://matplotlib.org/contents.html?v=20200131112331
    online seaborn documentation: https://seaborn.pydata.org/

Note that whenever you see multiple consecutive question marks (like '???') you will have to enter something. Evaluating a cell can be done by clicking on the 'run' button at the top or by pressing shift + enter on a selected cell.

Good luck!

## Quick Links to the Exercises

[Exercise 1 : Linear Regression](#exercise1)  
[Exercise 2 : Logistic Regression](#exercise2)  
[Exercise 3 : Decision Trees](#exercise3)  
[Exercise 4 : Random Forest](#exercise4)  
[Exercise 5 : K-Means Clustering](#exercise5)   
[Bonus      : Selecting Variables based on Importance](#bonus) 


# Import required packages #

#### Import packages
    import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor#, partial_dependence
from sklearn.inspection import plot_partial_dependence
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import auc, roc_curve, precision_recall_curve
from sklearn.dummy import DummyRegressor
from sklearn.cluster import KMeans
from sklearn import preprocessing

# for tree visualization
from io import StringIO # make sure this is installed?
from IPython.display import Image  
import pydotplus

sns.set_theme(style="darkgrid")

#### Load dataset Model_data.csv and check first rows

    pd.read_csv: Loads csv file
    DataFrame.head(): Shows first n rows of the table (default: 5)

In [ ]:
data = pd.read_csv('data/cleaned_dataset.csv')
print(data.shape)
data.head()


#### For our modeling, we only want some of the columns

The right columns have been decided with the process expert and the data scientist. This can however also be a guess in the beginning, and be updated when we start testing some models.

Let's use the following 6 columns as inputs:

    'glass_temp_zone1',
    'glass_temp_zone4',
    'temp_chamber17',
    'pressing_pressure',
    'pressing_time',
    'cycle_time',
    
And the target variable (the one we want to predict):

    'geometry_final'


In [ ]:
# Select only a couple of variables
variables = [
    'glass_temp_zone1',
    'glass_temp_zone4',
    'temp_chamber17',
    'pressing_pressure',
    'pressing_time',
    'cycle_time',
    'geometry_final',
    ]

# data_model = data[variables].copy()
data_model = data[?????].copy()

data_model = data_model.dropna()

data_model

# <a id='exercise1'>Exercise 1 - Linear Regression</a>

In this exercise, we will use a __linear regression__ model to see if we can predict 'geometry_final' from the predictor variables. 

In [ ]:
# As always, let's look at the data using describe()
data_model.describe()

#### Split the data into train and test data sets
    model_selection.train_test_split(): Returns 4 tables:
        X_train: modeling set for training
        Y_train: target set for training
        X_test: modeling set for testing
        Y_test: target set for testing

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    data_model.drop('geometry_final', axis=1),  # data set excl. target function
    data_model['geometry_final'],               # target function
    test_size=0.2,                                # ratio of train and test sets
    random_state=123                              # setting up root for random split of data
)

Take a look into X_train, X_test, Y_train and Y_test:

    DataFrame.head()
    DataFrame.shape

In [ ]:
# Explore the data of the train and test sets
X_train.shape

#### Create the linear regression object and train it
    LinearRegression(): Defines model to be used, in this case: LinearRegression from linear_model class
    fit(): Trains defined model on the training set

In [ ]:
regr = LinearRegression()       # Create the lR model called regr
regr.fit(X_train, Y_train);     # Train the model on the train datasets

#### The coefficients and intercept
    coef_: Lists coefficients of the linear regression model
    intercept_: Lists intercept of the linear regression model
    
Reminder: if we want to show more than 1 output per Jupyter cell, we need to use print() function.

In [ ]:
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

#### Make predictions using the testing set
    predict(): Applies trained model to new data

In [ ]:
pred = regr.predict(X_test)
pred

#### Evaluate the model
    mean_squared_error(actual value, predicted value)   : Calculates MSE between actual and predicted value
    r2_score(actual value, predicted value)             : Calculates R2 between actual and predicted value

In [ ]:
print("METRICS FOR LINEAR REGRESSION")
print('Test Set Mean Squared Error : ', mean_squared_error(Y_test, pred))
print('Test Set R2                 : ', r2_score(Y_test, pred))

#### Compare performance to a benchmark (default) model

To better appreciate if this performance is good or not, and if the model has learnt from the input variables, a good strategy is to compare it to a benchmark model that does not uses input variables.

    DummyRegressor(): defines an extremely simple model that does not use inputs. Default strategy is to always predicts the mean of the training set. This model behaves the same way as any other scikit-learn model, with functions .fit() and .predict().

In [ ]:
# create the benchmark model
dummy = DummyRegressor()

# train the model with input and output variables from the train set
dummy.fit(X_train, Y_train)

# compute predictions for the test set
Y_dummy = dummy.predict(X_test)

In [ ]:
print("METRICS FOR DUMMY REGRESSION")
print('Test Set Mean Squared Error : ', mean_squared_error(Y_test, Y_dummy))
print('Test Set R2                 : ', r2_score(Y_test, Y_dummy))

In [ ]:
# note that this can be equivalently done by:
np.mean((Y_test-np.mean(Y_train))**2)

In [ ]:
benchmark = mean_squared_error(Y_test, Y_dummy)
MSE = mean_squared_error(Y_test, pred)

print("The MSE has dropped by", 100*(benchmark-MSE)/benchmark, "% thanks to the model.")

In [ ]:
sns.scatterplot(x=Y_test, y=pred, label='Predictions')
sns.lineplot(x=Y_test, y=Y_test, color='gray', label='Perfect model')

plt.axis('equal') # Make both axes scaled equally
plt.xlabel('Real geometry_final [mm]')
plt.ylabel('Predicted geometry_final [mm]')

## Bonus: Plot residuals

In [ ]:
# Using MatPlotLib
sns.scatterplot(x=pred, y=(Y_test - pred), label='Predictions')
plt.axhline(0, color='gray', label='Perfect model') # adds reference line y = 0
plt.xlabel("Predicted values")
plt.ylabel("Rediduals")

# <a id='exercise2'>Exercise 2 - Logistic Regression</a>

#### Define new target variable
The quality control department has established that a produced piece is *good* if the target variable 'geometry_final' is above between -1.0 and 1.0. We will now try to predict if a piece is good from input variables, using classification models.

Let's define a new variable 'geometry_ok' which is 1 when the product is good (-1.0 > 'geometry_final' > 1.0) and 0 when the product is not good.

We'll start by looking at a line graph of 'geometry_final'.

#### Create a line plot of 'geometry_final' variable

In [ ]:
data_model['geometry_final'].plot()

# Plot lines where we're thinking about putting limits
plt.axhline(y=1.0, ls='--');
plt.axhline(y=-1.0, ls='--');

#### Define the new variable 'geometry_ok'
New variable is equal 1 if 'Rolling_force_avg' is above the threshold value 1.75, otherwise it is 0.
A recommended method to assign a value to a column only for a selection of rows is to use the .loc command:

    data.loc[condition, 'variable_to_assign'] =  value_to_assign

In [ ]:
# first we define a new column with value 0 for all rows
data_model['geometry_ok'] = 0

# then we assign the value 1 only on rows where 'geometry_final' is between -1.0 and 1.0
data_model.loc[(data_model['geometry_final'].between(-1, 1)), 'geometry_ok'] = 1

In [ ]:
# check that this is correct
data_model.head()

#### Count values of 'Target' variable
    DataFrame['variable'].value_counts(): Count number of rows per level of variable

In [ ]:
# How much of the data is in the desired range (1) and outside the desired range (0)
data_model['geometry_ok'].value_counts()

#### Split train and test
    train_test_split(): Returns 4 tables (modeling and target for train and test sets)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    data_model.drop(['geometry_final', 'geometry_ok'], axis=1), # data set excl. target function
    data_model['geometry_ok'],                                     # target function
    test_size=0.2,                                          # ratio of train and test sets
    random_state=0                                           # setting up root for random split of data
)

#### Create the logistic regression object and train it
    LogisticRegression(): Defines model to be used, in this case: Logistic regression from linear_model class
    fit(): Trains defined model

In [ ]:
#LogisticRegression?

In [ ]:
glm = LogisticRegression()
glm.fit(X_train, Y_train)

#### Make predictions using the testing set
    predict(): Applies trained model to new data

In [ ]:
# Predict the 'Target' using the test set
target_pred = glm.predict(X_test)
target_pred

In [ ]:
# compare with the real value
np.array(Y_test)

#### The coefficients and intercept
    coef_
    intercept_

In [ ]:
print('Coefficients: \n', glm.coef_)
print('Intercepts: \n', glm.intercept_)

#### Evaluate using the Confusion Matrix
    confusion_matrix(actual, predicted)  : Calculates confussion matrix for test actual and predicted values
    accuracy_score(actual, predicted)    : Calculates accuracy of the model (ratio of good answers)
    
    Learn more here : https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

In [ ]:
# The confusion matrix and metrics are classes we imported from SKLearn at the start 
print('Confusion matrix: \n', confusion_matrix(Y_test,target_pred))
print('Accuracy: ', accuracy_score(Y_test, target_pred))

conf_mat = confusion_matrix(Y_test,target_pred)

df_cm = pd.DataFrame(conf_mat, index=['Real not OK','Real OK'], columns=['Predicted not OK', 'Predicted OK'])
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d', cbar=False) # font size

### Bonus: threshold, precision/recall tuning, ROC and AUC

Play with the value of the probability threshold (default value: 0.5) to see the effect on recall and precision scores. 

- **accuracy**:  ratio of good answers (global) (TP+TN)/(TP+TN+FP+FN)
- **precision**: ratio of good answers among predictions of good products TP/(TP+FP)
- **recall**:    ratio of good answers among good products TP/(TP+FN)
- **F1 score**:  harmonic mean of precision and recall, decreases as soon as one of them decreases

If the priority is to avoid False Positives (selling an undetected bad product could be very dangerous), **precision** must be high.

If the priority is to avoid False Negatives (predictive maintenance where extra checks are not an issue but missing one is), **recall** must be high.

If both are important (quality control with high cost of production), **F1 score** must be high.

In [ ]:
glm = LogisticRegression()
glm.fit(X_train, Y_train)

# default value for the threshold is 0.5
# try with different values in [0, 1] to see the effect
threshold = 0.4

# prediction is replaced by a two-steps process
target_probas = glm.predict_proba(X_test)
target_pred = (target_probas[:,1]>=threshold).astype(int)

# compute and show the different metrics
print('Confusion matrix: \n', confusion_matrix(Y_test,target_pred))
print("Accuracy score:   {:.3f}".format(accuracy_score(Y_test, target_pred)))
print("Recall score:     {:.3f}".format(recall_score(Y_test,target_pred)))
print("Precision score:  {:.3f}".format(precision_score(Y_test,target_pred)))
print("F1 score:         {:.3f}".format(f1_score(Y_test,target_pred)))

Here is a more systematic search for the ideal threshold, where we plot the three metrics with respect to the threshold:

In [ ]:
precision, recall, thresholds = precision_recall_curve(Y_test, target_probas[:, 1]) 
F1 = 1/(0.5*(1/(recall+0.001)+1/(precision+0.001)))

plt.title("Precision-Recall vs Threshold Chart")
plt.plot(thresholds, precision[:-1], "b--", label="Precision")
plt.plot(thresholds, recall[:-1], "r--", label="Recall")
plt.plot(thresholds, F1[:-1], "k--", label="F1")
plt.ylabel("Precision, Recall, F1")
plt.xlabel("Threshold")
plt.legend(loc="lower left")
plt.ylim([0,1])

As could be expected, when the threshold increases (harder/tougher quality test), precision increases but recall decreases.

The better compromise (optimal F1) is obtained around threshold=0.4.

It is sometimes interesting to quantify the **global** performance of a classifier model, over any choices of threshold, to assess its ability to solve different problems. This is the purpose of **ROC curve** and **AUC score**.

The AUC is usually between 0.5 (bad score) and 1 (optimal score).

In [ ]:
Y_score = glm.decision_function(X_test)
fpr, tpr, _ = roc_curve(Y_test, Y_score)
roc_auc = auc(fpr, tpr)
print("AUC is", roc_auc)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# <a id='exercise3'>Exercise 3 - Decision Tree</a>

#### Create a decision tree object and train it
    
In this exercise we are going to use the Decision Tree for regression, using the continuous variable 'Rolling_force_avg' as target.  
    
    DecisionTreeRegressor(): Defines model to be used, in this case: Regression Tree from tree class
    max_depth: Parameter of a decision tree, defines how many levels with the tree have

In [ ]:
# Create our Train and Test data sets again.  
X_train, X_test, Y_train, Y_test = train_test_split(
    data_model.drop(['geometry_final', 'geometry_ok'], axis=1), # data set excl. target function
    data_model['geometry_final'],                                          # target function
    test_size=0.2,                                           # ratio of train and test sets
    random_state=123                                         # setting up root for random split of data
)

In [ ]:
# Create a Regression Tree model
# Constrain the max_depth and the min_samples_leagf
dt = DecisionTreeRegressor(max_depth=4, min_samples_leaf=5);
dt.fit(X_train, Y_train);

#### Visualize the tree
We'll use a packege called "graphviz" to draw the tree. Essentially, it's a special kind of graph.

In [ ]:
#Create the result object
dot_data = StringIO()

# This object will take the model (dt) and produce the elements necessary for the graph and put them in dot_data
export_graphviz(dt, out_file=dot_data, filled=True, rounded=True, special_characters=True, feature_names=X_train.columns)

#Create the graph frol dot_data
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

# Show the image
Image(graph.create_png())

#### Make predictions using the testing set
    predict(): Applies trained model to new data

In [ ]:
# Make the predictions using X_test
tree_pred = dt.predict(X_test)

#### Evaluate the model
    mean_squared_error(actual value, predicted value):  Calculated MSE between actual and predicted value
    r2_score(actual value, predicted value):            Calculates R2 between actual and predicted valu

In [ ]:
# Show the evaluation results for Test, but also for Train for comparison
print("METRICS FOR DECISION TREE")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, tree_pred))
print('Test Set R2                : ', r2_score(Y_test, tree_pred))
print('Train Set R2               : ', r2_score(Y_train, dt.predict(X_train)))

#### Plot prediction vs. actual
Choose matplotlib or seaborn

In [ ]:
sns.scatterplot(x=Y_test, y=tree_pred, label='Predictions')
sns.lineplot(x=Y_test, y=Y_test, color='gray', label='Perfect model')

plt.axis('equal') # Make both axes scaled equally
plt.xlabel('Real geometry_final [mm]')
plt.ylabel('Predicted geometry_final [mm]')

## Bonus: Plot residuals

In [ ]:
# Using MatPlotLib
sns.scatterplot(x=tree_pred, y=(Y_test - tree_pred), label='Prediction residuals')
plt.axhline(0, color='gray', label='Perfect model') # adds reference line y = 0
plt.xlabel("Predicted values")
plt.ylabel("Rediduals")

##  Bonus: Change the tree parameters
Dicision tree parameters can be changed in DecisionTreeRegressor definition, i.e. min. number of samples for a leaf.
Full breakdown of available options can be found here:   
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor

Modify parameters of your choice and evaluate new model performance.

In [ ]:
# Reprint the current RMSE and R2 (for comparison later) for : (max_depth=4, min_samples_leaf= 5)
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, tree_pred))
print('Test Set R2                : ', r2_score(Y_test, tree_pred))
print('Train Set R2               : ', r2_score(Y_train, dt.predict(X_train)))

In [ ]:
# Play with changing the depth of the tree
dt2 = DecisionTreeRegressor(max_depth=5);
dt2.fit(X_train, Y_train);
tree_pred2 = dt2.predict(X_test)

print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, tree_pred2))
print('Test Set R2                : ', r2_score(Y_test, tree_pred2))
print('Train Set R2               : ', r2_score(Y_train, dt2.predict(X_train)))

# <a id='exercise4'>Exercise 4 - Random Forest</a>

The challenge with a decision tree is overfitting, as seen from results of the previous exercice.  __Random Forest__ overcomes that by have a forest of trees which are randomly configured with different data.  

We will use the same data as before, so again, we are using Random Forest for __Regression__.

In [ ]:
# Create our Train and Test data sets again.  
X_train, X_test, Y_train, ????? = train_test_split(
    data_model.drop(['geometry_final', 'geometry_ok'], axis=1), # data set excl. target function
    data_model['geometry_final'],                          # target function
    test_size=0.2,                                           # ratio of train and test sets
    random_state=123                                         # setting up root for random split of data
)

#### Train the random forest
    RandomForestRegressor() : regression model using a random forest 

In [ ]:
# Create the Random Forest model
rf = RandomForestRegressor(n_estimators=100)

# train the model
rf.fit(X_train, Y_train)

Predict values of the response variable for new observations by the trained model using the other part of the data as test set.

In [ ]:
rf_pred = rf.predict(?????)

#### Check model accuracy
    mean_squared_error()
    r2_score()
    mean_absolute_error
We can use our wrapped function regression_validation for it as well!

In [ ]:
print("METRICS FOR RANDOM FOREST")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, rf_pred))
print('Test Set R2                : ', r2_score(?????, rf_pred))
print('Train Set R2               : ', r2_score(Y_train, rf.predict(X_train)))

#### Plot prediction vs. actual

In [ ]:
sns.scatterplot(x=Y_test, y=rf_pred, label='Predictions')
sns.lineplot(x=Y_test, y=Y_test, color='gray', label='Perfect model')

plt.axis('equal') # Make both axes scaled equally
plt.xlabel('Real geometry_final [mm]')
plt.ylabel('Predicted geometry_final [mm]')

#### Understaning the model with variable importance

With a normal decision tree, we can print out a graph of the rules used to make the regression.  That is not possible with Random Forest.  

In its place, we can find out which variables (features) have the most influence. This is a bit complicated at first, so don't panic!

    feature_importances_

In [ ]:
# Create a dataframe with two columns : Feature and Feature Importance
# Sort the dataframe by Feature Importance in ascending order
imp = pd.DataFrame({'Feature': X_train.columns,
                    'Feature Importance': rf.feature_importances_}). \
        sort_values('Feature Importance', ascending=True)

#Create a horizontal bar graph showing the features in their order of importance
plt.barh(range(len(imp)), imp['Feature Importance'], color='b', align='center')
plt.yticks(range(len(imp)), imp['Feature'])
plt.xlabel('Relative Importance')

# Before printing the bar graph, print theFeature Importance dataset
imp.sort_values('Feature Importance', ascending=False)

The above information will be useful when discussing the results of the model with the process expert.  While we don't know the exact rules, at least we know which features are considered most important by this model! 

#### Understanding variable importance and influence with partial dependence plots

A way to investigate how single input variables influence the target variable, we can use partial dependence plots. They do not exactly and quantitatively describe the relationship, but instead qualitatively how the model responds to changes in the input, keeping the rest of the variables fix. Let's analyze our Random Forest Model with this method:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

plot_partial_dependence(rf, 
                        X_train,
                        features=range(X_train.shape[1]), 
                        feature_names=X_train.columns,
                        grid_resolution=50,
                        ax=ax)

## Bonus: Tune the model adjusting its parameters
    n_estimators: number of trees
    max_features: # of variables
    max_depth: maximum depth of the three

In [ ]:
rf = RandomForestRegressor(
    n_estimators=200, 
    max_features=4, 
    max_depth=7)

rf.fit(X_train, Y_train)
rf_pred = rf.predict(X_test)

print("METRICS FOR RANDOM FOREST")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, rf_pred))
print('Test Set R2                : ', r2_score(Y_test, rf_pred))
print('Train Set R2               : ', r2_score(Y_train, rf.predict(X_train)))


## Bonus: alternative with GradientBoosting and study of partial dependences

In [ ]:
gbr = GradientBoostingRegressor(max_depth=5, learning_rate=0.03, min_samples_leaf=1, 
                                n_estimators=200, subsample=0.4, loss='lad')
gbr.???(X_train, Y_train)
gbr_pred = gbr.predict(X_test)
print("METRICS FOR GRADIENT BOOSTING")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, gbr_pred))
print('Test Set R2                : ', r2_score(Y_test, gbr_pred))
print('Train Set R2               : ', r2_score(Y_train, gbr.predict(X_train)))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

plot_partial_dependence(gbr, 
                        X_train,
                        features=range(X_train.shape[1]), 
                        feature_names=X_train.columns,
                        grid_resolution=50,
                        ax=ax)

## Bonus: alternative with GradientBoosting and study of partial dependences

In [ ]:
gbr = GradientBoostingRegressor(max_depth=5, learning_rate=0.03, min_samples_leaf=1, 
                                n_estimators=200, subsample=0.4, loss='lad')
gbr.???(X_train, Y_train)
gbr_pred = gbr.predict(X_test)
print("METRICS FOR GRADIENT BOOSTING")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, gbr_pred))
print('Test Set R2                : ', r2_score(Y_test, gbr_pred))
print('Train Set R2               : ', r2_score(Y_train, gbr.predict(X_train)))

In [ ]:
# use sklearn.inspection.plot_partial_dependence for more recent versions of sklearn

fig, ax = plt.subplots(figsize=(10, 10))

plot_partial_dependence(gbr, 
                        X_train,
                        features=range(X_train.shape[1]), 
                        feature_names=X_train.columns,
                        grid_resolution=50,
                        ax=ax)

## Bonus: Simplifying model on subset of data

For a fairly complex model, it seems like three variables in particular are important.

We've also seen in previous plots that the data differs quite a bit between different recipes. Insterad of trying to make a model for any case, it might make sense to make a model for one stable production state (=recipe)

Below, it looks like the latest production with Recipe-C is fairly homogenous:

In [ ]:
sns.scatterplot(data=data, x='geometry_final', y='glass_temp_zone1', hue='recipe')

In [ ]:
# Choose only the rows where the recipe columns says Recipe-C
data_regr = data[data['recipe'] ??? 'Recipe-C'].copy()

regr_features = ['glass_temp_zone1', 'pressing_pressure', 'pressing_time', 'geometry_final']
data_regr = data_regr[regr_features]
data_regr = data_regr.dropna()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    data_regr.drop('geometry_final', axis=1),  # data set excl. target function
    data_regr['geometry_final'],               # target function
    test_size=0.2,                                # ratio of train and test sets
    random_state=123                              # setting up root for random split of data
)

In [ ]:
regr = LinearRegression()       # Create the lR model called regr
regr.fit(X_train, Y_train);     # Train the model on the train datasets

In [ ]:
regr_pred = regr.predict(X_test)

In [ ]:
sns.scatterplot(x=Y_test, y=regr_pred, label='Predictions')
sns.lineplot(x=Y_test, y=Y_test, color='gray', label='Perfect model')

plt.axis('equal') # Make both axes scaled equally
plt.xlabel('Real geometry_final [mm]')
plt.ylabel('Predicted geometry_final [mm]')

print("METRICS FOR LINEAR REGRESSION")
print('Test Set Mean Sqared Error : ', mean_squared_error(Y_test, regr_pred))
print('Test Set R2                : ', r2_score(Y_test, regr_pred))
print('Train Set R2               : ', r2_score(Y_train, regr.predict(X_train)))

print(f'\nIntercept: \t{regr.intercept_:.3f}')
for f, c in zip(regr_features, regr.coef_):
    print(f'{f}: {c:.4f}')

#### For the latest data with Recipe-C, we could build a successfull simple linear model with good performance!

What we might have lost on accuracy and trying to model every process, we win in simplicity and intuition.

**For example, this model predicts that for an increase of 1 degree C in 'glass_temp_zone1' will lead to an 0.64 mm increase in 'geometry_final'.**

This can then be easily applied in different ways without using a black box model!

# <a id='exercise5'>Exercise 5 - K-means Clustering</a>

In this exercice we aim at clustering data points, which is a task for unsupervised machine learning. 
As a consequence, there is no need for spliting the data into train and test sets and the whole dataset will be use to train the model.

#### Load and visualize the data

In [ ]:
data_cluster = data_model[['geometry_final', 'glass_temp_zone1']].copy()

sns.scatterplot(data=data_cluster, x='geometry_final', y='glass_temp_zone1')

#### Fit Clusters for n=3
    KMeans(n_clusters=n): Defines model: k-means clustering with n clusters; and trains it using data
    fit(X): Trains defined model using data in table X
    
    Learn more here :   
    https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html  

In [ ]:
# Create the Kmeans model and train it
# We're not really training it.  We are just giving it the data that it will use.  
kmeans = KMeans(n_clusters=3)
kmeans.fit(data_cluster)

#### Make predictions using the testing set
    predict(): Applies trained model to new data to get predicted cluster allocations

In [ ]:
# Identify the clusters
y_kmeans = kmeans.predict(data_cluster)

#### Calculate centroids of clusters
    kmeans.cluster_centers_: Returns list of central points for the trained clusters

In [ ]:
# Get the centers from the model and print them
centers = kmeans.cluster_centers_
print(centers)

#### Plot cluster centroids and predicted clusters
Let's compose the chart drawing two scatter plots on one plot.

In [ ]:
#Create a first chart which are the data points, with colors showing the 3 clusters defined in y_kmeans3
sns.scatterplot(data=data_cluster, x='geometry_final', y='glass_temp_zone1', hue=y_kmeans)
#Create a second chart that shows the centroids in red
plt.scatter(centers[:,0], centers[:,1], color='r')
#plt.axis('equal')

**We can see that the clusters form pretty horizontal lines.** This is because the scale and variability of 'glass_temp_zone1' is bigger than that of 'geometry_final' => the distance in this dimension is bigger. 

(We can visualize this by uncommenting the line **#plt.axis('equal')** in the cell above.)

Let's try **normalizing** the data to see what happens.

In [ ]:
# Copy the data to make a normalized set
data_cluster_norm = data_cluster.copy()

# Calculate and subtract the mean from all values column-wise
data_cluster_norm = data_cluster_norm - data_cluster_norm.mean()

# Calculate and divide by the standard deviation for all values column-wise
data_cluster_norm = data_cluster_norm / data_cluster_norm.std()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(data_cluster_norm)

y_kmeans = kmeans.predict(data_cluster_norm)

centers = kmeans.cluster_centers_

sns.scatterplot(data=data_cluster_norm, x='geometry_final', y='glass_temp_zone1', hue=y_kmeans)
plt.scatter(centers[:,0], centers[:,1], color='r')
plt.axis('equal')

As a reference, let's see how the data was grouped after recipe:

In [ ]:
sns.scatterplot(data=data, x='geometry_final', y='glass_temp_zone1', hue='recipe')

#### Try to change the number of clusters from 3 to something else! What do we see?

#### Although we're only visualizing the clustering in two dimension, we can give the clustering model many dimension, for example the same data that we used for the previous models:

In [ ]:
# Use the same variable selection as for the previous exercises
data_cluster_all = data_model.copy()

# Normalize the variables (here we subtract the mean and divide by the standard deviation in one line!)
data_cluster_all = (data_cluster_all - data_cluster_all.mean()) / data_cluster_all.std()

kmeans = KMeans(n_clusters=3)
kmeans.fit(data_cluster_all)

y_kmeans = kmeans.predict(data_cluster_all)

centers = kmeans.cluster_centers_

sns.scatterplot(data=data_cluster_all, x='geometry_final', y='glass_temp_zone1', hue=y_kmeans)

#### Calculate inertia
    kmeans.inertia_: Measure of how internally coherent are the clusters

In [ ]:
print('Inertia: ', kmeans.inertia_)

Which number of clusters resulted in better classification (inertia was lower)?

## Bonus: Finding the optimum number of clusters: The Elbow method

How many clusters gives the best answer?  One technique is to test out several values and plot them on a curve.  The curve will have an elbow and that will indicate the best number of clusters.  

In [ ]:
# Run through a range of cluster values
# For each value, find the cluster centers and calculate the inertia

Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k, random_state=0)
    km = km.fit(data_cluster_norm)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
# Print a graph with x= Cluster count and Y = Inertia

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')

Which number of clusters would you choose?

# <a id='bonus'>Bonus - Selecting Variables based on Importance</a>

## Selection of variables from features importance

At the very start, we loaded our dataset and then selected only a subset of the variables to be used to predict Rolling_force_avg.

A more methodical way is to actually run a quick linear regression with penalization (Ridge model) to see which are the most influential variables and then focus on those.  

In this exercise, we'll see how that is done.  

In [ ]:
# make the train-test split with all numeric variables
cols = data_model.select_dtypes(include=float).columns

X_train, X_test, Y_train, Y_test = train_test_split(
    data_model[cols].drop('geometry_final', axis=1), # data set excl. target function
    data_model['geometry_final'],                       # target function
    test_size=0.2,                                      # ratio of train and test sets
    random_state=123                                    # setting up root for random split of data
)

In [ ]:
# look at performance for dummy regressor
dummy = DummyRegressor();
dummy.fit(X_train, Y_train);
Y_pred = dummy.predict(X_test)
print(mean_squared_error(Y_test, Y_pred))

In [ ]:
# look for a good regularization parameter alpha for the Ridge model 
for a in np.logspace(-4, -1, num=10):
    regr = Ridge(alpha=a, normalize=True)
    regr.fit(X_train, Y_train)
    Y_pred = regr.predict(X_test)
    print("alpha:", np.round(a, 4), "MSE:", np.round(mean_squared_error(Y_test, Y_pred), 3),
          "R2:", np.round(r2_score(Y_test, Y_pred), 4))

In [ ]:
# train the Ridge model
regr = Ridge(alpha=0.01, normalize=True)
regr.fit(X_train, Y_train)
Y_pred = regr.predict(X_test)
print("MSE:", mean_squared_error(Y_test, Y_pred))
print("R2:", r2_score(Y_test, Y_pred))

In [ ]:
# look at coefficients' importance, given by the product of 
# - the coefficient value
# - the std of the associated variable
# for more details, see:
# https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html

coefs = pd.DataFrame(regr.coef_ * X_train.std(axis=0),
                     columns=['Coefficient importance'], 
                     index=data_model[cols].drop('geometry_final', axis=1).columns)
coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model, small regularization')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)